In [ ]:
# 여기에 코드를 작성하세요.

# 환경 변수 로드
from dotenv import load_dotenv

load_dotenv()

# 필수 라이브러리 임포트
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import gradio as gr

# 메시지 플레이스홀더가 있는 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 여행 일정을 계획해주는 전문 AI 어시스턴트입니다. 사용자의 선호도, 예산, 일정에 맞춰 구체적이고 실용적인 여행 계획을 제공합니다.",
        ),
        MessagesPlaceholder("chat_history"),
        ("human", "{user_input}"),
    ]
)


# 사용자 메시지를 처리하고 AI 응답을 생성하는 함수 (chat_history 사용)
def answer_invoke(message, history, temperature):
    # Temperature에 따라 매번 새로운 모델 인스턴스 생성
    model = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=temperature,  # 사용자가 선택한 값 사용
        top_p=0.9,
        presence_penalty=0.3,
        frequency_penalty=0.3,
    )

    # 체인 생성
    chain = prompt | model | StrOutputParser()

    # Gradio history를 LangChain 메시지 형식으로 변환
    print(f"\n{'=' * 50}")
    print(f"📝 현재 입력 메시지: {message}")
    print(f"📚 History 개수: {len(history)}개")

    history_messages = []
    for msg in history:
        if msg["role"] == "user":
            # content가 리스트인 경우 텍스트만 추출
            content = msg["content"]
            if isinstance(content, list):
                text_content = " ".join(
                    [
                        item.get("text", "")
                        for item in content
                        if item.get("type") == "text"
                    ]
                )
                history_messages.append(HumanMessage(content=text_content))
            else:
                history_messages.append(HumanMessage(content=content))
        elif msg["role"] == "assistant":
            content = msg["content"]
            if isinstance(content, list):
                text_content = " ".join(
                    [
                        item.get("text", "")
                        for item in content
                        if item.get("type") == "text"
                    ]
                )
                history_messages.append(AIMessage(content=text_content))
            else:
                history_messages.append(AIMessage(content=content))

    # 변환된 히스토리 출력
    print(f"\n🔄 LangChain 형식으로 변환된 메시지:")
    for i, msg in enumerate(history_messages, 1):
        role = "👤 사용자" if isinstance(msg, HumanMessage) else "🤖 AI"
        content_preview = (
            msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
        )
        print(f"  {i}. {role}: {content_preview}")
    print(f"{'=' * 50}\n")

    # 체인 실행
    response = chain.invoke({"chat_history": history_messages, "user_input": message})
    return response


# 초기 환영 메시지 정의
initial_message = """안녕하세요! 🌍 맞춤형 여행 일정을 계획해드리는 AI 어시스턴트입니다.

완벽한 여행 계획을 위해 다음 정보를 알려주세요:

📍 **여행지**: 어디로 가고 싶으신가요?
📅 **여행 기간**: 며칠 동안 여행하실 계획인가요?
💰 **예산**: 대략적인 예산 범위는 어떻게 되시나요?
👥 **여행 인원**: 혼자, 가족, 친구들과 함께 가시나요?
🎯 **여행 스타일**: 휴양, 관광, 액티비티, 맛집 투어 등 선호하시는 스타일이 있나요?

위 정보를 자유롭게 말씀해주시면, 맞춤형 여행 일정을 상세히 계획해드리겠습니다! ✈️"""

# Gradio Blocks를 사용하여 커스텀 인터페이스 생성
with gr.Blocks(analytics_enabled=False) as demo:
    gr.Markdown("# 🌍 맞춤형 여행 일정 계획 어시스턴트")
    gr.Markdown(
        "여행지, 기간, 예산, 선호도를 알려주시면 맞춤형 여행 일정을 계획해드립니다!"
    )

    # Temperature 슬라이더 추가
    temperature_slider = gr.Slider(
        minimum=0.0,
        maximum=1.0,
        value=0.7,
        step=0.1,
        label="🎨 창의성 조절 (Temperature)",
        info="낮을수록(0.0) 일관되고 예측 가능한 답변, 높을수록(1.0) 창의적이고 다양한 답변을 생성합니다.",
        render=False,  # ChatInterface 내부에 표시
    )

    # 초기 메시지가 있는 Chatbot 생성 (최신 Gradio 형식)
    chatbot = gr.Chatbot(
        value=[{"role": "assistant", "content": initial_message}], height=500
    )

    # ChatInterface 생성
    gr.ChatInterface(
        fn=answer_invoke,
        chatbot=chatbot,  # 초기 메시지가 설정된 chatbot 사용
        additional_inputs=[temperature_slider],
        examples=[
            ["서울에서 2박 3일 여행 계획 짜줘", 0.7],
            ["제주도 가족 여행 일정 추천해줘", 0.3],
            ["유럽 배낭여행 1주일 계획 도와줘", 0.1],
        ],
    )

# Gradio 인터페이스 실행
demo.launch()


In [ ]:
# Gradio 인터페이스 종료
demo.close()